In [18]:
import pandas as pd
from sqlalchemy import create_engine

In [19]:
produkcja = pd.read_csv("export_produkcja.csv", encoding="utf-8-sig", sep=";")

produkcja = produkcja.applymap(lambda x: str(x).replace(",", ".") if isinstance(x, str) else x)

produkcja["Od"] = pd.to_datetime(produkcja["Od"], dayfirst=True, errors="coerce")
produkcja["Do"] = pd.to_datetime(produkcja["Do"], dayfirst=True, errors="coerce")

float_cols = [col for col in produkcja.columns if col not in ["Od", "Do", "Linia", "Rodzina"]]
produkcja[float_cols] = produkcja[float_cols].apply(pd.to_numeric, errors="coerce")

produkcja = produkcja[['Od', 'Do', 'Linia', 'Rodzina',
       'QL (TOT) Akt', 'QL (TOT) Pln', '% Dvt Produk.', 'Zmiana Czysty',
       'Zmiana Prg', 'Q/Zm. Akt', 'Q/Zm. Docel', 'Q/Zm. Standard', 'Q/CPK Akt',
       'Q/CPK Docel', 'Ope/LN Sh. Akt', 'Ope/LN Sh. Docel', 'Ope/EL Sh. Akt',
       'Ope/EL Sh. Docel', 'G/QL Akt', 'G/QL Docel', '% S.C. Eff',
       '% S.C. Std.', '% SR Eff', '% SR Std.', '% SF/SP Eff', '% SF/SP Std.',
       'Godz. Prac. Akt', 'Godz./Prac. Docel', '% Eff LINIA Eff',
       '% Eff LINIA Obb', '% Eff Prac. Eff', '% Eff Prac. Obb', '% Zysku Eff',
       '% Zysku Obb']]

produkcja.columns = [  
    "Od", "Do", "Linia", "Rodzina", "QLTOTAkt", "QLTOTPln", "ProcentDvtProduk",
    "ZmianaCzysty", "ZmianaPrg", "QZmAkt", "QZmDocel", "QZmStd",
    "QCPKAkt", "QCPKDocel", "OpeLNShAkt", "OpeLNShDocel",
    "OpeELShAkt", "OpeELShDocel", "GQLAkt", "GQLDocel",
    "ProcentSCEff", "ProcentSCStd", "ProcentSREff", "ProcentSRStd",
    "ProcentSFSPEff", "ProcentSFSPStd", "GodzPracAkt", "GodzPracDocel",
    "ProcentELINIAEff", "ProcentELINIAObb", "ProcentEPracEff", "ProcentEPracObb",
    "ProcentZyskuEff", "ProcentZyskuObb"
]

produkcja.drop_duplicates(subset=["Od", "Linia", "Rodzina"], inplace=True)

server = 'JASNACZERN'        
database = 'ferrero'
username = 'sa'
password = '1234'
driver = 'ODBC Driver 17 for SQL Server'

conn_str = (
    f"mssql+pyodbc://{username}:{password}@{server}/{database}"
    f"?driver={driver.replace(' ', '+')}"
)
engine = create_engine(conn_str)

try:
    produkcja.to_sql("BilansProdukcji", con=engine, if_exists="append", index=False)
    print("Data inserted into BilansProdukcji table.")
except Exception as e:
    print("Insert failed:", e)

produkcja.columns

Data inserted into BilansProdukcji table.


Index(['Od', 'Do', 'Linia', 'Rodzina', 'QLTOTAkt', 'QLTOTPln',
       'ProcentDvtProduk', 'ZmianaCzysty', 'ZmianaPrg', 'QZmAkt', 'QZmDocel',
       'QZmStd', 'QCPKAkt', 'QCPKDocel', 'OpeLNShAkt', 'OpeLNShDocel',
       'OpeELShAkt', 'OpeELShDocel', 'GQLAkt', 'GQLDocel', 'ProcentSCEff',
       'ProcentSCStd', 'ProcentSREff', 'ProcentSRStd', 'ProcentSFSPEff',
       'ProcentSFSPStd', 'GodzPracAkt', 'GodzPracDocel', 'ProcentELINIAEff',
       'ProcentELINIAObb', 'ProcentEPracEff', 'ProcentEPracObb',
       'ProcentZyskuEff', 'ProcentZyskuObb'],
      dtype='object')